In [1]:
#Imports
import json 
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from datetime import datetime
from os import listdir


#Set seed to ensure experiment replication
SEED = 1234
tf.random.set_seed(SEED)

# Get current working directory
cwd = os.getcwd()


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#Connect to Drive to fetch data
from google.colab import drive
drive.mount('/content/drive') 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#unzip the data
!unzip '/content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip'

Output streaming troncato alle ultime 5000 righe.
  inflating: MaskDataset/training/17141.jpg  
  inflating: __MACOSX/MaskDataset/training/._17141.jpg  
  inflating: MaskDataset/training/11530.jpg  
  inflating: __MACOSX/MaskDataset/training/._11530.jpg  
  inflating: MaskDataset/training/15756.jpg  
  inflating: __MACOSX/MaskDataset/training/._15756.jpg  
  inflating: MaskDataset/training/14448.jpg  
  inflating: __MACOSX/MaskDataset/training/._14448.jpg  
  inflating: MaskDataset/training/12987.jpg  
  inflating: __MACOSX/MaskDataset/training/._12987.jpg  
  inflating: MaskDataset/training/13441.jpg  
  inflating: __MACOSX/MaskDataset/training/._13441.jpg  
  inflating: MaskDataset/training/15030.jpg  
  inflating: __MACOSX/MaskDataset/training/._15030.jpg  
  inflating: MaskDataset/training/16539.jpg  
  inflating: __MACOSX/MaskDataset/training/._16539.jpg  
  inflating: MaskDataset/training/11256.jpg  
  inflating: __MACOSX/MaskDataset/training/._11256.jpg  
  inflating: MaskDatase

In [5]:
#Create Data Generators
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Training data augmentation to training data Toggle
apply_data_augmentation = False

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        validation_split=0.2,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split=0.2)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)




dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir= os.path.join(dataset_dir,"training")

#Batch size
BS = 8

#Image shape
IMG_H = 407 #612
IMG_W = 407

#Number of classes
num_classes=3

#Create training and validation generators from ImageDataGenerators
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic=json.load(f)
  dataframe= pd.DataFrame(dic.items())
  dataframe.rename(columns={0:'filename', 1:'class'}, inplace=True )
  dataframe=dataframe.sample(frac=1, random_state=SEED)
  dataframe["class"]=dataframe["class"].astype('string')
  train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle= True,
                                                 seed=SEED,
                                                 subset='training',
                                                 image_size=(IMG_H,IMG_W))
  
  validation_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle= True,
                                                 seed=SEED,
                                                 subset='validation',
                                                 image_size=(IMG_H,IMG_W))
  




Found 4492 validated image filenames belonging to 3 classes.
Found 1122 validated image filenames belonging to 3 classes.


In [6]:
#create Datasets Objects

#Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

#Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: validation_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()


In [7]:
#VGG model setup
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMG_H, IMG_W, 3))
vgg.summary()

58892288/58889256 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 407, 407, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 407, 407, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 407, 407, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 203, 203, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 203, 203, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 203, 203, 128)     147584    
______________________________________________________________

In [8]:
# Architecture: VGG -> Classifier

#Finetuning model
finetuning = True

if finetuning:
    freeze_until = 7 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False

#Create model
model = tf.keras.Sequential() #we implement this in a sequential way 
model.add(vgg)

#Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=407, activation='relu'))
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))


In [9]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 12, 12, 512)       14714688  
_________________________________________________________________
flatten (Flatten)            (None, 73728)             0         
_________________________________________________________________
dense (Dense)                (None, 407)               30007703  
_________________________________________________________________
dropout (Dropout)            (None, 407)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               52224     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 44,775,002
Trainable params: 44,514,842
Non-trainable params: 260,160
______________________________________

[<tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
 array([[[[ 4.29470569e-01,  1.17273867e-01,  3.40129584e-02, ...,
           -1.32241577e-01, -5.33475243e-02,  7.57738389e-03],
          [ 5.50379455e-01,  2.08774377e-02,  9.88311544e-02, ...,
           -8.48205537e-02, -5.11389151e-02,  3.74943428e-02],
          [ 4.80015397e-01, -1.72696680e-01,  3.75577137e-02, ...,
           -1.27135560e-01, -5.02991639e-02,  3.48965675e-02]],
 
         [[ 3.73466998e-01,  1.62062630e-01,  1.70863140e-03, ...,
           -1.48207128e-01, -2.35300660e-01, -6.30356818e-02],
          [ 4.40074533e-01,  4.73412387e-02,  5.13819456e-02, ...,
           -9.88498852e-02, -2.96195745e-01, -7.04357103e-02],
          [ 4.08547401e-01, -1.70375049e-01, -4.96297423e-03, ...,
           -1.22360572e-01, -2.76450396e-01, -3.90796512e-02]],
 
         [[-6.13601133e-02,  1.35693997e-01, -1.15694344e-01, ...,
           -1.40158370e-01, -3.77666801e-01, -3.00509870e-01],
    

In [10]:
#Optimization parameters:

# Loss function
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [11]:
#create Callbacks

cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/Keras/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'First'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

#Implementation of Early Stopping
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)
    callbacks.append(es_callback)

In [12]:
#Fit model 
model.fit(x=train_dataset,
          epochs=1,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(validation_gen),
          callbacks=callbacks)

562/562 [==============================] - 300s 534ms/step - loss: 1.1003 - accuracy: 0.3315 - val_loss: 1.0985 - val_accuracy: 0.3333


In [13]:
#Make prediction on test data and export it as csv file

#Create_csv function definition
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

#import test data
test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

#create predictions
predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1


Found 450 validated image filenames belonging to 1 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
57/57 [==============================] - 19s 333ms/step


In [14]:
#create csv file
create_csv(results, '/content/drive/My Drive')